In [1]:
def bitvect_to_list(bitvect):
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(bitvect, arr)
    return arr.tolist()

In [2]:
import pandas as pd
casmi2022 = pd.read_pickle('casmi2022_remove_01_2010_4606_pos.p')
casmi2016 = pd.read_pickle('casmi2016_remove_01_2010_4606_pos.p')
casmi2017 = pd.read_pickle('casmi2017_remove_01_2010_4606_pos.p')

/Users/zhangxinran/opt/miniconda3/envs/metfid/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
db5 = pd.read_csv('DB5_v1.2_new.csv')
db5['CANONICAL_SMILES'] = db5['CANONICAL_SMILES'].fillna(db5['canonical_smiles'])

/var/folders/fw/5y38398x4pgb4ks5np5s62tw0000gn/T/ipykernel_3809/3665815543.py:1: DtypeWarning: Columns (0,1,3,6,8,12,13,15,18,21,22,23,24,25,26,27,31,33,48) have mixed types. Specify dtype option on import or set low_memory=False.
  db5 = pd.read_csv('DB5_v1.2_new.csv')


In [4]:
performance_df_MLP = pd.read_pickle('saved_bins_performance_positive.p')
performance_df_MLP = pd.DataFrame(performance_df_MLP)
top_500_bins_MLP = performance_df_MLP.sort_values(by='f1_score_val', ascending=False).head(500)
selected_bins_MLP = top_500_bins_MLP['bin_index'].tolist()

In [5]:
casmi2022['Spectrum Vector'] = casmi2022['Spectrum Vector'].apply(lambda x: x[selected_bins_MLP])
casmi2016['Spectrum Vector'] = casmi2016['Spectrum Vector'].apply(lambda x: x[selected_bins_MLP])
casmi2017['Spectrum Vector'] = casmi2017['Spectrum Vector'].apply(lambda x: x[selected_bins_MLP])

casmi2022 = casmi2022.dropna(subset=['fp_decode'])
casmi2016 = casmi2016.dropna(subset=['fp_decode'])
casmi2017 = casmi2017.dropna(subset=['fp_decode'])

In [5]:
# casmi2016['fp_decode'] = casmi2016['fp_decode'].apply(bitvect_to_list)
# casmi2016['fp_decode'] = casmi2016['fp_decode'].apply(lambda x: x if isinstance(x, list) and len(x)==2048 else None)
# casmi2016 = casmi2016.dropna(subset=['fp_decode'])

In [6]:
def tanimoto_coefficient(y_true, y_pred):
    y_pred_rounded = tf.round(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred_rounded)
    sum_true = tf.reduce_sum(y_true)
    sum_pred = tf.reduce_sum(y_pred_rounded)
    union = sum_true + sum_pred - intersection
    tanimoto = intersection / union
    return tanimoto

def sokal_sneath_3(y_true, y_pred):
    y_pred_rounded = tf.round(y_pred)
    
    a = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred_rounded, 1)), tf.float32))
    b = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred_rounded, 0)), tf.float32))
    c = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred_rounded, 1)), tf.float32))
    d = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred_rounded, 0)), tf.float32))
    
    score = (1/4) * ((a / (a + b)) + (a / (a + c)) + (d / (b + d)) + (d / (c + d)))
    return score

import tensorflow as tf
positive_model = tf.keras.models.load_model('best_model_2010_4606_positive_remove_01_lstm.h5', custom_objects={'tanimoto_coefficient': tanimoto_coefficient, 'sokal_sneath_3':sokal_sneath_3})

In [39]:
import pickle
import numpy as np

with open("f1_scores_per_fingerprint_4606_positive.p", 'rb') as f:
    full_fingerprints_f1 = pickle.load(f)

selected_fingerprints = np.where(full_fingerprints_f1 >= 0.85)[0]

In [40]:
def predict_fingerprint(binned_vector, loaded_model):
    binned_vector_np = np.array(binned_vector)
    binned_vector_np = binned_vector_np.reshape(1, -1)
    predicted = loaded_model.predict(binned_vector_np, verbose=0)[0]
    # with open("selected_fingerprints.p", 'rb') as f:
    #     selected_fingerprints = pickle.load(f)
    predicted = predicted[selected_fingerprints]
    return np.round(predicted).astype(int)

In [41]:
casmi2022['fp_predicted'] = casmi2022['Spectrum Vector'].apply(lambda x: predict_fingerprint(x, positive_model))
casmi2016['fp_predicted'] = casmi2016['Spectrum Vector'].apply(lambda x: predict_fingerprint(x, positive_model))
casmi2017['fp_predicted'] = casmi2017['Spectrum Vector'].apply(lambda x: predict_fingerprint(x, positive_model))

In [10]:
db5.rename(columns={'inchikey':'InChIKey'}, inplace=True)

In [102]:
db5.rename(columns={'InChIKey':'InChIkey'}, inplace=True)

In [11]:
casmi2022 = pd.merge(casmi2022, db5[['InChIKey', 'canonical_smiles']], on='InChIKey', how='left')

In [103]:
casmi2017 = pd.merge(casmi2017, db5[['InChIkey', 'canonical_smiles']], on='InChIkey', how='left')

In [22]:
casmi2022_predicted_priority = pd.read_csv('2022_prority_formula_identifications_all.tsv', sep='\t')
casmi2022_predicted_bonus = pd.read_csv('2022_bonus_formula_identifications_all.tsv', sep='\t')
casmi2016_predicted = pd.read_csv('2016_formula_identifications_all.tsv', sep='\t')
casmi2017_predicted = pd.read_csv('2017_formula_identifications_all.tsv', sep='\t')

In [13]:
# casmi2022_predicted = pd.read_csv('mist_cf_casmi2022_positive.tsv', sep='\t')
casmi2022_predicted = pd.concat([casmi2022_predicted_priority, casmi2022_predicted_bonus])

In [11]:
from rdkit import Chem
from rdkit.Chem import AllChem

def compute_morgan_fingerprint(smiles, radius=2, n_bits=2048):
    if not isinstance(smiles, str):  # Check if input is a string
        return None  # Return None or some default value for invalid inputs
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return fingerprint
    else:
        return None

In [16]:
# casmi2022.loc[(casmi2022['canonical_smiles'].isnull()==False), 'fp_decode'] = casmi2022.loc[(casmi2022['canonical_smiles'].isnull()==False), 'canonical_smiles'].apply(compute_morgan_fingerprint)
# casmi2022.loc[(casmi2022['canonical_smiles'].isnull()==False), 'fp_decode'] = casmi2022.loc[(casmi2022['canonical_smiles'].isnull()==False), 'fp_decode'].apply(bitvect_to_list)

In [14]:
def calculate_adduct(pm, mode):
    pos = {'M+H': pm - 1.007276, 'M+NH4': pm - 18.033823, 'M+Na': pm - 22.989218}
    neg = {'M-H': pm + 1.007276, 'M+Cl': pm - 34.969402, 'M+FA-H': pm - 44.998201}
    if mode == 'positive':
        pm_table = [i for i in pos.values()]
        return pm_table
    elif mode == 'negative':
        pm_table = [i for i in neg.values()]
        return pm_table
    else:
        raise ValueError("The ion mode should be 'positive' or 'negative'.")

In [15]:
def calculate_tanimoto(real_fp, predicted_fp):
    """
    Given a compound fingerprint and a predicted fingerprint, calculate the
    tanimoto score.
    :param real_fp: candidate's fingerprint
    :param predicted_fp: predicted fingerprint
    :return: tanimoto score
    """
    predicted_fingerprint = list(map(lambda x: int(x), predicted_fp))
    fp_length = len(predicted_fingerprint)

    if fp_length != len(real_fp):
        raise ValueError('Different size of vector!')

    identity_num = 0
    length = 0
    for i in range(fp_length):
        if predicted_fingerprint[i] == real_fp[i] == 1:
            identity_num += 1
            length += 1
        elif predicted_fingerprint[i] != real_fp[i]:
            length += 1
    # print(identity_num / length)
    return identity_num / length

In [43]:
import numpy as np
from tabulate import tabulate
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

def min_max_scale(group):
    scaler = MinMaxScaler()
    scaled_scores = scaler.fit_transform(group.values.reshape(-1, 1))
    return scaled_scores.ravel()

def retrieve_compound(fingerprint, database, fp_predicted, challenge_id, mass_list, ppm):
    compound_dict = {}
    
    fp_predicted['challenge_id'] = fp_predicted['id'].apply(lambda x:int(x.split('_')[2]))
    # fp_predicted = fp_predicted[fp_predicted['spec'] == challenge_id-81]
    fp_predicted['SiriusScore'] = fp_predicted.groupby('challenge_id')['SiriusScore'].transform(min_max_scale)
    # fp_predicted['scores'] = fp_predicted.groupby('spec')['scores'].transform(min_max_scale)
    candidate_dict = retrieve_candidate(database, mass_list, ppm)
    tanimoto = []
    f1 = []
    try:
        for candidate, fp in candidate_dict.items():
            tanimoto.append(calculate_tanimoto(fp, fingerprint))
            f1.append(f1_score(fp, fingerprint))
            compound_dict[candidate] = calculate_tanimoto(fp, fingerprint)
            if fp_predicted[fp_predicted['molecularFormula'] == candidate[2]].shape[0]>0:
                # print(fp_predicted[fp_predicted['molecularFormula'] == candidate[2]]['SiriusScore'])
                compound_dict[candidate] = 0.8*calculate_tanimoto(fp, fingerprint)+0.2*float(fp_predicted[fp_predicted['molecularFormula'] == candidate[2]]['SiriusScore'].iloc[0])
                # compound_dict[candidate] = 0.01*calculate_tanimoto(fp, fingerprint)+1.5*float(fp_predicted[fp_predicted['cand_form'] == candidate[2]]['scores'])
            else:
                compound_dict[candidate] = calculate_tanimoto(fp, fingerprint)
    except AttributeError:
        print("Compound Not Found!")

    return (compound_dict, np.mean(tanimoto), np.mean(f1))

def pos_1_decode(lst):
    result = [0] * 7293
    for i in lst:
        result[i] = 1

    return result


def retrieve_candidate(database, mass_list, ppm):
    """
    Given a search type, a database, mass(optional), and inchikey(optional)
    returns a dict that contains all qualified candidates.
    :param database: database.csv
    :param mass_list: list of precursor mass
    :param ppm: mass tolerance in ppm
    :return: dict{(Name, Inchikey): candidate fingerprint}

    REQ: The search type must be one of 'mass' or 'formula'. For now, the
    database is not provided by Ressom's lab, and it must be a .csv file.
    """
    df = database
    candidate_dict = {}
    for mass in mass_list:
        min_weight = mass * 1000000.0 / (1000000.0 + ppm)
        max_weight = mass * 1000000.0 / (1000000.0 - ppm)
        candidate_df = df[(df['monoisotopic_mass'] >= min_weight) & (df['monoisotopic_mass'] <= max_weight)]
        if len(candidate_df) == 0:
            continue
        # candidate_df['morgan_fp'] = candidate_df['CANONICAL_SMILES'].apply(compute_morgan_fingerprint)
        candidate_list = candidate_df[['name', 'InChIKey', 'formula', 'fingerprint']].values.tolist()
        # candidate_list = candidate_df[['name', 'InChIkey2D', 'molecularFormula', 'fingerprint']].values.tolist()
        with open('fp_filtered_4606.p', 'rb') as f:
            fp_indexes_list = pickle.load(f)
        # with open('_files/fp_to_add_indexes_new.p', 'rb') as f:
        #     fp_indexes_5618 = pickle.load(f)

        for candidate in candidate_list:
            # new_fp = [full_length_fp[i] for i in range(len(full_length_fp)) if i not in fp_indexes_list]
            if candidate[3] is None:
                continue
            fp = candidate[3]
            # fp = candidate[2]
            try:
            #     # fp = fp.strip('[]').split(', ')
                fp = [int(i) for i in fp.strip('[]').split(', ')]
                fp = pos_1_decode(fp)
            except:
                continue
            new_fp = [fp[i] for i in range(len(fp)) if i in fp_indexes_list]
            fp = new_fp
            fp = [fp[i] for i in selected_fingerprints]
            fp = [int(i) for i in fp]
            candidate_dict[candidate[0], candidate[1], candidate[2]] = fp

    return candidate_dict

In [17]:
def visualize_compound_dict(compound_dict, compound_name=True):
    """
    Given a compound dict, prints out the dict as a table in an order of
    descending tanimoto scores.
    :param compound_dict: compound dict
    :param compound_name: if we can obtain the compound name
    :return: None
    """
    sort_list = []
    sorted_dict = {k: v for k, v in sorted(compound_dict.items(), key=lambda item: item[1], reverse=True)}

    if compound_name:
        for k, v in sorted_dict.items():
            sort_list.append([str(k[0]), str(k[1]), str(k[2]), str(v)])
            # sort_list.append([str(k[0]), str(k[1]), str(v)])

        return tabulate(sort_list, headers=['Compound Name', 'Inchikey', 'Formula', 'Score'])
        # return tabulate(sort_list, headers=['Compound Name', 'Formula', 'Score'])
    else:
        for k, v in sorted_dict.items():
            sort_list.append([str(k), str(v)])

        return tabulate(sort_list, headers=['Inchikey', 'Score'])
        # return tabulate(sort_list, headers=['Formula', 'Score'])

In [18]:
db5.loc[:207, 'monoisotopic_mass'] = db5.loc[:207, 'PRECURSOR_MZ']
db5['monoisotopic_mass'] = pd.to_numeric(db5['monoisotopic_mass'])

In [20]:
casmi2022 = casmi2022[casmi2022['canonical_smiles'].isnull()==False]

In [105]:
casmi2017 = casmi2017[casmi2017['canonical_smiles'].isnull()==False]

In [28]:
casmi2016.columns

Index(['SourceFile', 'ChallengeName', 'PRECURSOR_MZ', 'ION_MODE', 'RT',
       'nPeaks', 'NAME', 'SMILES', 'INCHI', 'InChIKey', 'CSID', 'PC_CID',
       'mass', 'intensity', 'Formula', 'CANONICAL_SMILES', 'fingerprint',
       'mass_intensity_pairs', 'Mass', 'Intensity', 'Spectrum Vector',
       'fp_decode', 'fp_predicted'],
      dtype='object')

In [45]:
casmi2022.columns

Index(['Compound Number', 'File', 'RT [min]', 'Precursor m/z (Da)',
       'Priority/Bonus', 'Adduct', 'InChIKey', 'SMILES', 'Collection',
       'Rack ID or UCD Barcode', 'Well ID', 'Formula', 'Monoisotopic Mass',
       'Mix or Set Number', 'fingerprint', 'mass', 'intensity',
       'mass_intensity_pairs', 'Mass', 'Intensity', 'Spectrum Vector',
       'fp_decode', 'fp_predicted', 'canonical_smiles'],
      dtype='object')

In [21]:
for i in range(casmi2022.shape[0]):
    predicted_fp = casmi2022.iloc[i]['fp_predicted']
    precursor = calculate_adduct(float(casmi2022.iloc[i]['Precursor m/z (Da)']), 'positive')
    compound_dict = retrieve_compound(predicted_fp, db5, casmi2022_predicted, casmi2022.iloc[i]['Compound Number'], precursor, 5)
    output_file = 'mist-cf_casmi2022_lstm_test/' + 'Challenge-' + str(casmi2022.iloc[i]['Compound Number']) + '_prediction.txt'
    with open(output_file, 'a') as result:
        result.write(str(casmi2022.iloc[i]['Compound Number']) + '\n')
        result.write(visualize_compound_dict(compound_dict[0]) + '\n')

In [44]:
for i in range(casmi2016.shape[0]):
    predicted_fp = casmi2016.iloc[i]['fp_predicted']
    precursor = calculate_adduct(float(casmi2016.iloc[i]['PRECURSOR_MZ']), 'positive')
    compound_dict = retrieve_compound(predicted_fp, db5, casmi2016_predicted, int(casmi2016.iloc[i]['ChallengeName'].split('-')[1]), precursor, 5)
    output_file = 'mist-cf_casmi2016_lstm_test/' + casmi2016.iloc[i]['ChallengeName'] + '_prediction.txt'
    with open(output_file, 'a') as result:
        result.write(str(casmi2016.iloc[i]['ChallengeName']) + '\n')
        result.write(visualize_compound_dict(compound_dict[0]) + '\n')

In [111]:
for i in range(casmi2017.shape[0]):
    predicted_fp = casmi2017.iloc[i]['fp_predicted']
    precursor = calculate_adduct(float(casmi2017.iloc[i]['PRECURSOR_MZ']), 'positive')
    compound_dict = retrieve_compound(predicted_fp, db5, casmi2017_predicted, int(casmi2017.iloc[i]['Challenge'].split('-')[1]), precursor, 5)
    output_file = 'mist-cf_casmi2017_lstm_test/' + 'C' + casmi2017.iloc[i]['Challenge'][1:] + '_prediction.txt'
    with open(output_file, 'a') as result:
        result.write(str(casmi2017.iloc[i]['Challenge']) + '\n')
        result.write(visualize_compound_dict(compound_dict[0]) + '\n')

/Users/zhangxinran/opt/miniconda3/envs/metfid/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/zhangxinran/opt/miniconda3/envs/metfid/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/zhangxinran/opt/miniconda3/envs/metfid/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/zhangxinran/opt/miniconda3/envs/metfid/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/var/folders/fw/5y38398x4pgb4ks5np5s62tw0000gn/T/ipykernel_51186/3785653424.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [252]:
casmi2016 = pd.read_csv('casmi2016_solution_full_fingerprint.csv')

In [237]:
casmi2017 = pd.read_csv('casmi2017_solution_full_fingerprint.csv')

In [231]:
casmi2017 = casmi2017[casmi2017['Formula'].isnull()==False]

In [238]:
casmi2017.rename(columns={'InChIKey':'InChIkey'}, inplace=True)

In [239]:
casmi2017 = pd.merge(casmi2017, db5[['InChIkey', 'formula']], on='InChIkey', how='left')

In [253]:
casmi2016 = casmi2016[casmi2016['ION_MODE']==' POSITIVE']

In [261]:
casmi2022 = pd.read_csv('casmi2022_solution_full_fingerprint.csv')

In [259]:
casmi2022 = casmi2022[casmi2022['File'].str.contains('neg')]

In [262]:
correct_predictions = 0  # Counter for correct predictions

for i in range(casmi2022.shape[0]):
    # predicted_fp = casmi2016.iloc[i]['fp_predicted']
    precursor = calculate_adduct(float(casmi2022.iloc[i]['Precursor m/z (Da)']), 'negative')
    
    # Ensure the challenge_id column exists and is properly calculated
    casmi2022_predicted['challenge_id'] = casmi2022_predicted['id'].apply(lambda x: int(x.split('_')[2]))
    
    # Filter predicted formulas for the current challenge
    # challenge_id = int(casmi2022.iloc[i]['ChallengeName'].split('-')[1])
    challenge_id = int(casmi2022.iloc[i]['Compound Number'])
    predicted_formulas = casmi2022_predicted[casmi2022_predicted['challenge_id'] == challenge_id]
    
    # Ensure no missing SiriusScore
    predicted_formulas = predicted_formulas.dropna(subset=['SiriusScore'])
    
    # Find the highest SiriusScore
    max_score = predicted_formulas['SiriusScore'].max()
    
    # Get all rows where SiriusScore equals max_score
    highest_score_rows = predicted_formulas[predicted_formulas['SiriusScore'] == max_score]
    
    # Compare with the actual formula
    actual_formula = casmi2022.iloc[i]['Formula']  # Normalize case and remove extra spaces
    correct_in_this_challenge = False
    
    # Check if any predicted molecular formula matches the actual formula
    for _, row in highest_score_rows.iterrows():
        predicted_formula = row['molecularFormula']  # Normalize case and remove extra spaces
        if predicted_formula == actual_formula:
            correct_in_this_challenge = True
            break  # Stop if we found a match
    
    if correct_in_this_challenge:
        correct_predictions += 1  # Increment if the predicted formula matches the actual formula

# Final count of correct predictions as a percentage
accuracy = correct_predictions / casmi2022.shape[0]
print(f"Accuracy of correct formula predictions: {accuracy * 100:.2f}%")

Accuracy of correct formula predictions: 43.49%
